# Mel-Cepstral Distortion

This notebook will show you how to calculate the Mel-cepstral distortion (MCD) of target reference and converterted synthesised wavs that are not time aligned.

In [6]:
import os
import math
import glob
import librosa
import pyworld
import pysptk
import numpy as np
import matplotlib.pyplot as plot
import glob

## 0 Functions and Constants

In [7]:
def load_wav(wav_file, sr):  # wav file -> audio time series numpy array
    """
    Load a wav file with librosa.
    :param wav_file: path to wav file
    :param sr: sampling rate
    :return: audio time series numpy array
    """
    wav, _ = librosa.load(wav_file, sr=sr, mono=True)

    return wav


def log_spec_dB_dist(x, y):
    log_spec_dB_const = 10.0 / math.log(10.0) * math.sqrt(2.0)
    diff = x - y
    
    return log_spec_dB_const * math.sqrt(np.inner(diff, diff))


SAMPLING_RATE = 22050
FRAME_PERIOD = 5.0

## 1 The Data

The data used in this example is from LJSpeech and KSS using 99 and 78 sentences each.<br/>
LJSpeech was synthesized with [DeepVoice3](https://github.com/generative-model/voice_generation/tree/main/deepvoice3_pytorch) and KSS was synthesized with [Tacotron2](https://github.com/generative-model/voice_generation/tree/main/Tacotron2-Wavenet-Korean-TTS)
 * LJSpeech (en): https://keithito.com/LJ-Speech-Dataset/
 * KSS (kr) : https://www.kaggle.com/bryanpark/korean-single-speaker-speech-dataset

```
official_dir/wavs
    |
    |- audio_origin
        |- <TRG_ID>.wav
        |- ...
    |- audio_syn
        |- syn_<TRG_ID>.wav
        |- ...
```
 

## 2 Extract Acoustic Features

From the StarGAN-VC2 paper:

> Each speaker has sets of 81 and 35 sentences for training and evaluation, respectively. The recordings were downsampled to 22.05 kHz for this challenge. We extracted 34 Mel-cepstral coefficients (MCEPs), logarithmic fundamental frequency (log F0), and aperiodicities (APs) every 5 ms by using the WORLD analyzer [52].

* MCD: 6.9 +- .08 dB for StarGAN-VC
* MCD: 6.9 +- .07 dB for StarGAN-VC2
* Link: http://www.kecl.ntt.co.jp/people/kaneko.takuhiro/projects/stargan-vc2/index.html

Extract features using SPTK Mel-cepstral Analysis of speech (more info: http://sp-tk.sourceforge.net/)

This notebook uses python wrappers of the WORLD vocoder and SPTK to extract acoustic features:

* To get the spectral envelope: `pyworld` https://github.com/JeremyCCHsu/Python-Wrapper-for-World-Vocoder 
* To extract MCEP features: `pysptk` https://github.com/r9y9/pysptk  

In [9]:
def wav2mcep_numpy(wavfile, target_directory, alpha=0.65, fft_size=512, mcep_size=34): # .wav -> Mel-cepstral(.npy)
    # make relevant directories
    if not os.path.exists(target_directory):
        os.makedirs(target_directory)

    loaded_wav = load_wav(wavfile, sr=SAMPLING_RATE)

    # Use WORLD vocoder to spectral envelope
    _, sp, _ = pyworld.wav2world(loaded_wav.astype(np.double), fs=SAMPLING_RATE,
                                   frame_period=FRAME_PERIOD, fft_size=fft_size)

    # Extract MCEP features
    mgc = pysptk.sptk.mcep(sp, order=mcep_size, alpha=alpha, maxiter=0,
                           etype=1, eps=1.0E-8, min_det=0.0, itype=3)

    fname = os.path.basename(wavfile).split('.')[0]
    np.save(os.path.join(target_directory, fname + '.npy'),
            mgc,
            allow_pickle=False)

In [10]:
alpha = 0.65  # commonly used at 22050 Hz
fft_size = 512
mcep_size = 34


deep_origin_wavs = glob.glob('./audio_deep/wavs/audio_origin/*') # wavs/audio_origin/ 안 모든 wav files
deep_origin_mcep_dir = './audio_deep/mceps_numpy/audio_origin' # mceps_numpy/audio_origin 경로 
deep_syn_wavs = glob.glob('./audio_deep/wavs/audio_syn/*')
deep_syn_mcep_dir = './audio_deep/mceps_numpy/audio_syn'

taco_origin_wavs = glob.glob('./audio_taco/wavs/audio_origin/*') 
taco_origin_mcep_dir = './audio_taco/mceps_numpy/audio_origin' 
taco_syn_wavs = glob.glob('./audio_taco/wavs/audio_syn/*')
taco_syn_mcep_dir = './audio_taco/mceps_numpy/audio_syn'


for wav in deep_origin_wavs: # file 내의 모든 .wav -> mcep_numpy
    wav2mcep_numpy(wav, deep_origin_mcep_dir, fft_size=fft_size, mcep_size=mcep_size)

for wav in deep_syn_wavs:
    wav2mcep_numpy(wav, deep_syn_mcep_dir, fft_size=fft_size, mcep_size=mcep_size)

for wav in taco_origin_wavs:
    wav2mcep_numpy(wav, taco_origin_mcep_dir, fft_size=fft_size, mcep_size=mcep_size)

for wav in taco_syn_wavs:
    wav2mcep_numpy(wav, taco_syn_mcep_dir, fft_size=fft_size, mcep_size=mcep_size)

# 3 Calculate MCD

* Here I use librosa to calculate dynamic time warping (DTW) as it got a MCD score closer to that reported in the paper

In [11]:
def average_mcd(ref_mcep_files, synth_mcep_files, cost_function):
    """
    Calculate the average MCD.
    :param ref_mcep_files: list of strings, paths to MCEP target reference files
    :param synth_mcep_files: list of strings, paths to MCEP converted synthesised files
    :param cost_function: distance metric used
    :returns: average MCD, total frames processed
    """
    min_cost_tot = 0.0
    frames_tot = 0
    
    for ref in ref_mcep_files:
        for synth in synth_mcep_files:
            ref_id = os.path.basename(ref)
            synth_id = os.path.basename(synth).split('_',1)[-1]
            # if the speaker name is the same and sample id is the same, do MCD
            if ref_id == synth_id:
                # load MCEP vectors
                ref_vec = np.load(ref)
                ref_frame_no = len(ref_vec)
                synth_vec = np.load(synth)

                # dynamic time warping using librosa
                min_cost, _ = librosa.sequence.dtw(ref_vec[:, 1:].T, synth_vec[:, 1:].T, 
                                                   metric=cost_function)               
                min_cost_tot += np.mean(min_cost)
                frames_tot += ref_frame_no
                
    mean_mcd = min_cost_tot / frames_tot
    
    return mean_mcd, frames_tot

In [12]:
deep_origin_refs = glob.glob('./audio_deep/mceps_numpy/audio_origin/*')
deep_syn_synths = glob.glob('./audio_deep/mceps_numpy/audio_syn/*')
taco_origin_refs = glob.glob('./audio_taco/mceps_numpy/audio_origin/*')
taco_syn_synths = glob.glob('./audio_taco/mceps_numpy/audio_syn/*')

cost_function = log_spec_dB_dist

deep_mcd, deep_tot_frames_used = average_mcd(deep_origin_refs, deep_syn_synths, cost_function)
taco_mcd, taco_tot_frames_used = average_mcd(taco_origin_refs, taco_syn_synths, cost_function)


print(f'deepvoice MCD = {deep_mcd} dB, calculated over a total of {deep_tot_frames_used} frames')
print(f'tacotron MCD = {taco_mcd} dB, calculated over a total of {taco_tot_frames_used} frames')

deepvoice MCD = 7.780104466000925 dB, calculated over a total of 133460 frames
tacotron MCD = 6.416187556584367 dB, calculated over a total of 60339 frames
